In [1]:
import xarray
from dask.distributed import Client
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import xarray
import pandas as pd

from preprocess.sza import solarzenithangle


In [5]:

client = Client(n_workers = 12)

In [2]:
hres = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr')
# hres = hres.sel(lat=slice(None, 62)).drop_vars(('lat_bnds', 'lon_bnds', 'record_status'))

In [9]:
sis_reindex.SIS[2,:,:].values

array([[  0.,   0.,   0., ..., 399., 401., 400.],
       [  0.,   0.,   0., ..., 400., 401., 399.],
       [  0.,   0.,   0., ..., 404., 400., 400.],
       ...,
       [  0.,   0.,   0., ...,   1.,   1.,   1.],
       [  0.,   0.,   0., ...,   1.,   1.,   1.],
       [  0.,   0.,   0., ...,   1.,   1.,   1.]], dtype=float32)

In [ ]:
hres.to_zarr('HRSEVIRI.zarr', mode='w')

In [4]:
sis = xarray.open_mfdataset('/scratch/snx3000/kschuurm/DATA/SARAH3/SIS_*.nc', 
                            concat_dim='time', 
                            combine='nested', 
                            data_vars='minimal', 
                            coords='minimal', 
                            compat='override', 
                            engine='h5netcdf')
sis = sis.drop_duplicates(dim='time').drop_vars(['lat_bnds', 'lon_bnds'])

intersec_tidx = sorted(
        list(
            set(hres.time.dt.round("30min").values).intersection(
                set(sis.time.dt.round("30min").values)
            )
        )
    )


hres_tidx_sort = hres.time.copy().sortby(hres.time)
hres_intersec_tidx = hres_tidx_sort.sel(time=intersec_tidx, method='nearest')

sis_reindex = sis.reindex(time=hres_intersec_tidx, lat=hres.lat, lon=hres.lon, method='nearest')
sis_reindex = sis_reindex.reindex(time=hres.time, fill_value=np.nan)
sis_reindex = sis_reindex.chunk({'time':60, 'lat':-1, 'lon':-1})
# sis_reindex.to_zarr('HRSEVIRI.zarr', mode='a')


In [11]:
sis_reindex.to_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr', mode='a')

/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/xarray/core/dataset.py:2497: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(  # type: ignore[call-overload,misc]
/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


In [ ]:
cal = xarray.open_mfdataset(['/scratch/snx3000/kschuurm/DATA/SARAH3/CAL_2014.nc','/scratch/snx3000/kschuurm/DATA/SARAH3/CAL_2015.nc'] , 
                            concat_dim='time', 
                            combine='nested', 
                            data_vars='minimal', 
                            coords='minimal', 
                            compat='override', 
                            engine='h5netcdf')

cal = cal.drop_duplicates(dim='time').drop_vars(['lat_bnds', 'lon_bnds','record_status'])

In [ ]:
intersec_tidx = sorted(
        list(
            set(hres.time.dt.round("30min").values).intersection(
                set(sis.time.dt.round("30min").values)
            )
        )
    )


hres_tidx_sort = hres.time.copy().sortby(hres.time)
hres_intersec_tidx = hres_tidx_sort.sel(time=intersec_tidx, method='nearest')

cal_reindex = cal.reindex(time=hres_intersec_tidx, lat=hres.lat, lon=hres.lon, method='nearest')
cal_reindex = cal_reindex.reindex(time=hres.time, fill_value=np.nan)
cal_reindex = cal_reindex.chunk({'time':60, 'lat':-1, 'lon':-1})
cal_reindex.to_zarr('HRSEVIRI.zarr', mode='a')


In [ ]:
DEM = xarray.open_dataset('/scratch/snx3000/kschuurm/DATA/SRTM/SRTM.nc')
roi= (62,35,-15,28)
SRTM = DEM.SRTM.isel(lat=(DEM.lat >= roi[1])&(DEM.lat<= roi[0]), lon=(DEM.lon >= roi[2])&(DEM.lon <= roi[3]))

weight_lat = int(np.ceil(len(SRTM.lat) / len(hres.lon)))
weight_lon = int(np.ceil(len(SRTM.lon)/ len(hres.lon)))
SRTM_coarse = SRTM.coarsen(lon=weight_lon, boundary='trim').mean().coarsen(lat=weight_lat, boundary='trim').mean()
SRTM_coarse = SRTM_coarse.reindex(lat=hres.lat, lon=hres.lon, method='nearest')

SRTM_coarse.attrs = {'long_name': 'Digital Elevation Map of the Shuttle Radar Topography Mission',
                     'standard_name': 'DEM_SRTM',
                     'units': 'm',
                    }

SRTM_coarse.to_zarr('HRSEVIRI.zarr',mode='a')

In [2]:
hres = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr')

In [3]:
datetimes = pd.to_datetime(hres.time)


def solarzenithangle_latlon(da_temp):
    subsample_int = 4
    
    
    a = [x for x in range(0, len(da_temp.time), 1)]
    a.append(len(da_temp.time)-1)
    datetimes = pd.to_datetime(da_temp.time[a])
    
    lats = np.arange(da_temp.lat.min(), da_temp.lat.max()+1, 1, dtype=np.float32)
    lons = np.arange(da_temp.lon.min(), da_temp.lon.max()+1, 1, dtype=np.float32)
    
    da_sza = xarray.DataArray(coords={'time':datetimes, 'lat':lats, 'lon':lons,},
                          data=np.zeros(shape=(len(datetimes), len(lats), len(lons)),
                                       dtype=np.float16))
    da_sza.name = 'SZA'
    da_sza.attrs.update({'long_name': 'Solar Zenith Angle at sea level',
                      'standard_name': 'solar_zenith_angle',
                      'units':'rad'})
    da_azi = xarray.DataArray(coords={'time':datetimes, 'lat':lats, 'lon':lons,},
                          data=np.zeros(shape=(len(datetimes), len(lats), len(lons)), 
                                        dtype=np.float16))
    da_azi.name = 'AZI'
    da_azi.attrs.update({'long_name': 'Solar Azimuth Angle at sea level',
                      'standard_name': 'solar_azimuth_angle',
                      'units':'rad'})
    
    for i, lat in tqdm(enumerate(lats)):
        for j, lon in enumerate(lons):
            sza, azi = solarzenithangle(datetimes, lat, lon, 0)
            da_sza[:,i, j] = sza
            da_azi[:,i,j] = azi
    
    
    ds = xarray.Dataset({'SZA':da_sza, 'AZI':da_azi})
    return ds


        

In [ ]:
ds_sun = solarzenithangle_latlon(hres.SIS)

13it [08:47, 40.43s/it]

In [ ]:
ds_sun.chunk({'time':1, 'lat':-1, 'lon':-1}).to_zarr('temp.zarr')

In [ ]:
temp = xarray.open_zarr('temp.zarr')

In [ ]:
ds_sun_interp = temp.interp(lat=hres.lat, lon=hres.lon)

In [ ]:
temp2 = ds_sun_interp.to_zarr('temp2.zarr')

In [ ]:
for var in temp2:
    del temp2[var].encoding['chunk']

ds_sun_final = temp2.chunk({'time':60, 'lat':-1, 'lon':-1})

In [ ]:
ds_sun_final.to_zarr('/scratch/snx3000/kschuurm/DATA/HRSEVIRI.zarr', mode='a')
